In [253]:
import os
import matplotlib.pyplot as plt
os.chdir('/home/sarvagya/Invoice-AI')

In [254]:
from pdf2image import convert_from_path
import os

def pdf_to_png(pdf_path, output_folder, dpi=300):
    """
    Convert a single-page PDF to PNG.

    Args:
        pdf_path (str): Path to the input PDF file.
        output_folder (str): Folder where the output PNG file will be saved.
        dpi (int, optional): DPI (dots per inch) setting for the output image. Default is 300.
    """
    # Convert PDF to images
    images = convert_from_path(pdf_path, dpi=dpi)
    
    # Check if the PDF has at least one page
    if not images:
        print(f"No pages found in the PDF: {pdf_path}")
        return
    
    # Use the first page (assuming the PDF has only one page)
    image = images[0]
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Define the output file path
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    output_path = os.path.join(output_folder, f"{base_name}.png")
    
    # Save the image as PNG
    image.save(output_path, 'PNG')
    print(f"Image saved to {output_path}")

for file_name in os.listdir('test_data'):
    
    # Example usage
    pdf_path = f'test_data/{file_name}'
    output_folder = 'output_images'
    pdf_to_png(pdf_path, output_folder)


Image saved to output_images/sample2.png
Image saved to output_images/sample1.png
Image saved to output_images/sample 4.png
Image saved to output_images/sample 3.png


In [255]:
import cv2
import json

def get_contours(img_file_name):
    # Load the image
    image = cv2.imread(f'output_images/{img_file_name}.png')

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to get a binary image
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize a list to store contour information
    contour_info = []

    # Iterate over contours
    for i, contour in enumerate(contours):
        # Calculate the bounding box coordinates
        x, y, w, h = cv2.boundingRect(contour)
        
        # Calculate the center point
        center_x = x + w // 2
        center_y = y + h // 2
        
        # Append contour information to the list
        contour_info.append({
            'contour_index': i,
            'left_bottom': (x, y + h),  # Add height to y-coordinate to get bottom
            'top_right': (x + w, y),
            'center': (center_x, center_y),
            'bbox': (x, y, w, h)
        })
    basename = os.path.splitext(os.path.basename(img_file_name))[0]
    with open(f'bbox_data/original_contours/{basename}.json', 'w') as json_file:
        json.dump(contour_info, json_file, indent=4)

# Function to merge two bounding boxes
def merge_bboxes(bbox1, bbox2):
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    x = min(x1, x2)
    y = min(y1, y2)
    w = max(x1 + w1, x2 + w2) - x
    h = max(y1 + h1, y2 + h2) - y
    return (x, y, w, h)

def group_horizontal_contours(file_name):
    with open(f'bbox_data/original_contours/{file_name}.json', 'r') as json_file:
        contour_info = json.load(json_file)
    groups = {}
    group_count = 0

    # Assign groups to bounding boxes based on y-coordinate differences
    for i, contour in enumerate(contour_info):
        assigned = False
        for group in groups:
            for idx in groups[group]:
                other_contour = contour_info[idx]
                # Check y-coordinate difference
                if abs(contour['center'][1] - other_contour['center'][1]) < 30:
                    groups[group].append(i)
                    assigned = True
                    break
            if assigned:
                break
        if not assigned:
            group_count += 1
            groups[group_count] = [i]

    # Sort the values for each key based on the x-coordinate of the center of each bounding box
    for group in groups:
        groups[group].sort(key=lambda idx: contour_info[idx]['center'][0])

    # Save group information to a JSON file
    with open(f'bbox_data/grouped_contours/{file_name}.json', 'w') as json_file:
        json.dump(groups, json_file, indent=4)

    print("Group information saved to 'groups.json'.")
    
def merge_contours(file_name):
    with open(f'bbox_data/original_contours/{file_name}.json', 'r') as json_file:
        contour_info = json.load(json_file)
    with open(f'bbox_data/grouped_contours/{file_name}.json', 'r') as json_file:
        groups = json.load(json_file)
    
    # Function to merge two bounding boxes
    def merge_bboxes(bbox1, bbox2):
        x1, y1, w1, h1 = bbox1
        x2, y2, w2, h2 = bbox2
        x = min(x1, x2)
        y = min(y1, y2)
        w = max(x1 + w1, x2 + w2) - x
        h = max(y1 + h1, y2 + h2) - y
        return (x, y, w, h)
    
    # Padding size
    padding = 7
    
    # Initialize a list to store merged bounding boxes
    merged_bboxes = []

    # Merge bounding boxes for each group based on x-coordinate distance
    for group in groups:
        indices = groups[group]
        if not indices:
            continue
        # Start with the first bounding box in the group
        current_bbox = contour_info[indices[0]]['bbox']
        for i in range(1, len(indices)):
            next_bbox = contour_info[indices[i]]['bbox']
            # Check the distance between the right x-coordinate of the current bbox and the left x-coordinate of the next bbox
            if next_bbox[0] - (current_bbox[0] + current_bbox[2]) < 40:
                # Merge the bounding boxes
                current_bbox = merge_bboxes(current_bbox, next_bbox)
            else:
                # Pad the current merged bbox before appending
                current_bbox = (current_bbox[0] - padding, current_bbox[1] - padding, current_bbox[2] + 2 * padding, current_bbox[3] + 2 * padding)
                # Append the current merged bbox and move to the next
                merged_bboxes.append(current_bbox)
                current_bbox = next_bbox
        # Pad the last merged bbox before appending
        current_bbox = (current_bbox[0] - padding, current_bbox[1] - padding, current_bbox[2] + 2 * padding, current_bbox[3] + 2 * padding)
        # Append the last bbox
        merged_bboxes.append(current_bbox)

    # Save merged bounding box information to a JSON file
    merged_bboxes_info = [{'bbox': bbox} for bbox in merged_bboxes]
    with open(f'bbox_data/merged_contours/{file_name}.json', 'w') as json_file:
        json.dump(merged_bboxes_info, json_file, indent=4)

    print("Merged bounding box information saved to 'merged_bboxes.json'.")


In [256]:
import cv2
import matplotlib.pyplot as plt

def plot_bboxes(file_name, save_path='images_with_bbox/'):
    """
    Plot bounding boxes on the image.

    Args:
        image_path (str): Path to the input image.
        bboxes (list of dict): List of bounding boxes. Each bounding box is a dictionary with 'bbox' key.
        save_path (str, optional): Path to save the image with bounding boxes. If None, the image will not be saved.
    """
    
    with open(f'bbox_data/merged_contours/{file_name}.json', 'r') as json_file:
        bboxes = json.load(json_file)
    # Load the image
    image_path = f'output_images/{file_name}.png'
    image = cv2.imread(image_path)
    
    # Check if the image was loaded successfully
    if image is None:
        print(f"Failed to load image at {image_path}")
        return
    
    # Draw bounding boxes on the image
    for bbox_info in bboxes:
        x, y, w, h = bbox_info['bbox']
        # Draw rectangle on the image
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Convert image to RGB for displaying with matplotlib
    # image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # # Display the image with bounding boxes
    # plt.figure(figsize=(10, 10))
    # plt.imshow(image_rgb)
    # plt.axis('off')
    # plt.show()
    
    # Save the image with bounding boxes if save_path is provided
    if save_path:
        cv2.imwrite(f'{save_path}/{file_name}.png', image)
        print(f"Image with bounding boxes saved at {save_path}")

In [257]:
for file_name in os.listdir('output_images'):
    file_name = file_name.split('.')[0]
    get_contours(file_name)
    group_horizontal_contours(file_name)
    merge_contours(file_name)
    plot_bboxes(file_name)

Group information saved to 'groups.json'.
Merged bounding box information saved to 'merged_bboxes.json'.
Image with bounding boxes saved at images_with_bbox/
Group information saved to 'groups.json'.
Merged bounding box information saved to 'merged_bboxes.json'.
Image with bounding boxes saved at images_with_bbox/
Group information saved to 'groups.json'.
Merged bounding box information saved to 'merged_bboxes.json'.
Image with bounding boxes saved at images_with_bbox/
Group information saved to 'groups.json'.
Merged bounding box information saved to 'merged_bboxes.json'.
Image with bounding boxes saved at images_with_bbox/


### Reading Text from Image

In [258]:
from PIL import Image, ImageEnhance, ImageFilter

# Function to preprocess the image
def preprocess_image_high_resolution(image):
    # Increase resolution
    high_res_image = image.resize((image.width * 3, image.height * 3), Image.Resampling.LANCZOS)
    # Apply Gaussian Blur to smooth edges
    blurred_image = high_res_image.filter(ImageFilter.GaussianBlur(radius=1))
    # Convert to grayscale
    grayscale_image = blurred_image.convert("L")
    # Increase contrast
    enhancer = ImageEnhance.Contrast(grayscale_image)
    enhanced_image = enhancer.enhance(3)
    # Apply binarization
    binary_image = enhanced_image.point(lambda x: 0 if x < 128 else 255, '1')
    # Apply sharpening filter
    sharpened_image = binary_image.filter(ImageFilter.SHARPEN)
    return sharpened_image


from PIL import Image

from PIL import Image

def resize_with_aspect_ratio(image, target_width):
    """
    Resize an image while preserving its aspect ratio.

    Args:
        image (PIL.Image.Image): Input image.
        target_width (int): Target width for resizing.

    Returns:
        PIL.Image.Image: Resized image.
    """
    # Get the original width and height
    original_width, original_height = image.size
    
    # Calculate the scaling factor based on the target width
    scaling_factor = target_width / original_width
    
    # Calculate the new dimensions with the preserved aspect ratio
    new_width = int(original_width * scaling_factor)
    new_height = int(original_height * scaling_factor)
    
    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    
    return resized_image

def zoom_out(image, zoom_out_level=2):

    # Get the original image dimensions
    original_width, original_height = image.size
    # print(image.size)
    # Calculate new dimensions for zoomed-out image
    zoomed_out_width = original_width * zoom_out_level
    zoomed_out_height = original_height * zoom_out_level

    # Create a new image with white background for zooming out
    zoomed_out_image = Image.new(image.mode, (zoomed_out_width, zoomed_out_height), color='white')

    # Calculate the position to paste the original image centered
    paste_x = (zoomed_out_width - original_width) // 2
    paste_y = (zoomed_out_height - original_height) // 2

    # Paste the original image onto the zoomed-out image
    zoomed_out_image.paste(image, (paste_x, paste_y))

    # Resize back to the original dimensions
    resized_image = resize_with_aspect_ratio(zoomed_out_image, 300)

    # Save or display the resized image
    # resized_image.show()  # Display the image
    # resized_image.save('resized_image.png')  # Save the image
    # print(resized_image.size)
    return resized_image

# # Example usage
# zoom_out('output.png', zoom_out_level=5)

In [259]:
import re


def only_special_characters(text):
    # Define a regular expression pattern to match special characters
    # re.compile(r'^[!@#$%^&*(),.?":{}|<>[\]\\;\'/`~\-_=+’]+$')
    pattern = re.compile(r'^[!@#$%^&*(),.?":{}|<>[\]\\;\'/`~\-_=+’]+$')
    
    # Check if the entire text matches the pattern
    if pattern.fullmatch(text):
        return True
    else:
        return False
    
only_special_characters('’')

True

In [260]:
import json
import pytesseract
import re 

def only_special_characters(text):
    # Define a regular expression pattern to match special characters
    pattern = re.compile(r'^[!@#$%^&*(),.?":{}|<>[\]\\;\'/`~\-_=+’]+$')
    
    # Check if the entire text matches the pattern
    if pattern.fullmatch(text):
        return True
    else:
        return False

# Function to extract text from a bounding box
def extract_text_from_bbox(image:Image, bbox , lang='eng'):
    x, y, w, h = bbox
    cropped_image = image.crop((x, y, x + w, y + h))
    text = pytesseract.image_to_string(cropped_image, lang=lang).strip()
    if text == '' or only_special_characters(text):
        cropped_image = preprocess_image_high_resolution(cropped_image)
        text = pytesseract.image_to_string(cropped_image, lang=lang).strip()

    if text=='' or only_special_characters(text):
        cropped_image = zoom_out(cropped_image, zoom_out_level=3)
        text = pytesseract.image_to_string(cropped_image, lang=lang).strip()
        if text=='' or only_special_characters(text):
            custom_config = r'--oem 3 --psm 6 outputbase digits'
            cropped_image = zoom_out(cropped_image, zoom_out_level=3)
            text = pytesseract.image_to_string(cropped_image, config=custom_config).strip()
            if text=='':
                plt.figure(figsize=(10, 10))
                plt.imshow(cropped_image)
                plt.axis('off')
                plt.show()
    return text.strip()

# Extract text from each bounding box and store in a list
extracted_texts = []

with open('bbox_data/merged_contours/sample 3.json' , 'r')  as json_file:
    bbox_coordinates = json.load(json_file)

image = Image.open('output_images/sample 3.png')

for i ,bbox in enumerate(bbox_coordinates):
    bbox = bbox["bbox"]
    text = extract_text_from_bbox(image, bbox, lang='eng')
    extracted_texts.append([i, text])

# Print the extracted texts
for text in extracted_texts:
    print(text)

[0, 'BIC XXX12345, IBAN DE12387654321']
[1, 'Zahlungsempfanger\nBankverbindung']
[2, 'Max Mustermann\nDeutsche Bank AG']
[3, 'Max Mustermann']
[4, 'Mit freundlichen Gruen']
[5, 'Bitte begleichen Sie den offenen Betrag bis zum 01.07.2023 an die unten aufgefuhrte\nBankverbindung.']
[6, 'Vielen Dank fur Ihren Auftrag!']
[7, 'Rechnungsbetrag']
[8, '127,33 €']
[9, 'Nettobetrag 107,00 €\nUmsatzsteuer 19% 20,33 €']
[10, '']
[11, '2']
[12, '5678']
[13, 'Weiterberechnung Reinigungsmittel']
[14, '1']
[15, '17,00']
[16, '17,00 €']
[17, '1']
[18, '1234']
[19, 'Unkraut entfernt und Rase gemaht']
[20, '4,5 Std.']
[21, '20,00']
[22, '90,00 €']
[23, 'Pos']
[24, 'Art-Nr.']
[25, 'Bezeichnung']
[26, 'Menge']
[27, 'Einzelpreis']
[28, 'Betrag']
[29, '']
[30, 'Das Rechnungsdatum entspricht dem Leistungsdatum']
[31, 'Rechnung 202210001']
[32, 'Sehr geehrte Frau Beispiel,\nnachfolgend berechnen wir Ihnen wie vorab besprochen:']
[33, 'Rechnung']
[34, '202210001']
[35, 'Kunde']
[36, '123456']
[37, 'Datum']
[38,

In [261]:
from llama_index.core import PromptTemplate
from templates import FIND_TABLE_COLS, PRODUCT_DETAILS
from llm_utils import use_llm

FIND_TABLE_COLS = '''
You are provided with the list of word/sentences seperated... They are extracted from invoice or other similar documents.


List of words/sentences:
{extracted_data}


INSTRUCTIONS : 
You need to return the word whose name is similar to amount for given quantity and price.

ANSWER : 
'''

def contains_numbers(text):
    # Regular expression to match any digit
    pattern = r'\d+'
    
    # Search for the pattern in the text
    if re.search(pattern, text):
        return True
    else:
        return False
    
def get_total_col_of_table(extracted_texts:list):
    cols = []
    for x in extracted_texts:
        value = x[1]
        
        if len(value.split())<3 and not contains_numbers(value):
            cols.append(value)
    
    extracted = '\n-'.join(cols)
    prompt = PromptTemplate(FIND_TABLE_COLS).format(extracted_data = extracted) 
    response , _ = use_llm(prompt)
    
    print(response)
    split_index = len(extracted_texts)-1
    while split_index>=0:
        val = extracted_texts[split_index][1]
        if val.strip() == response.strip():
            break
        split_index-=1
        
    table_content , details_content = extracted_texts[:split_index+1] , extracted_texts[split_index+1:]
    return table_content , details_content

table_content , details_content = get_total_col_of_table(extracted_texts)
    

Betrag


In [262]:
import ast
PRODUCT_DETAILS = '''
You have been provided with data extracted from invoice tables. 

EXTRACTED DATA :
{extracted_data}

---------------------------------------------------
INSTRUCTIONS : 
- Return list of dictionaries for each product in the table
- Each dictionaries with keys as (description/product_name) , (quantity) , (unit_price) , (amount_for_quantity) and corresponding values as value of dict...
- Maintain the currency signs if present

- The bill/ total amount should also be included in the list along with taxes if present.
    For example :     [{{"Total" : "value"}} , {{"Taxes" : "value"}}]

ANSWER : 
'''

def get_table_data(table_content):
    extracted_data = ''
    for i in range(len(table_content)):
        extracted_data+=f"- {table_content[i][1]}\n"
    prompt = PromptTemplate(PRODUCT_DETAILS).format(extracted_data = extracted_data)
    response, _ = use_llm(prompt)
    print(prompt)
    print(response)
    return ast.literal_eval(response)
    
product_details = get_table_data(table_content)
print(product_details)


You have been provided with data extracted from invoice tables. 

EXTRACTED DATA :
- BIC XXX12345, IBAN DE12387654321
- Zahlungsempfanger
Bankverbindung
- Max Mustermann
Deutsche Bank AG
- Max Mustermann
- Mit freundlichen Gruen
- Bitte begleichen Sie den offenen Betrag bis zum 01.07.2023 an die unten aufgefuhrte
Bankverbindung.
- Vielen Dank fur Ihren Auftrag!
- Rechnungsbetrag
- 127,33 €
- Nettobetrag 107,00 €
Umsatzsteuer 19% 20,33 €
- 
- 2
- 5678
- Weiterberechnung Reinigungsmittel
- 1
- 17,00
- 17,00 €
- 1
- 1234
- Unkraut entfernt und Rase gemaht
- 4,5 Std.
- 20,00
- 90,00 €
- Pos
- Art-Nr.
- Bezeichnung
- Menge
- Einzelpreis
- Betrag


---------------------------------------------------
INSTRUCTIONS : 
- Return list of dictionaries for each product in the table
- Each dictionaries with keys as (description/product_name) , (quantity) , (unit_price) , (amount_for_quantity) and corresponding values as value of dict...
- Maintain the currency signs if present

- The bill/ total amo

In [263]:
details_content

[[29, ''],
 [30, 'Das Rechnungsdatum entspricht dem Leistungsdatum'],
 [31, 'Rechnung 202210001'],
 [32,
  'Sehr geehrte Frau Beispiel,\nnachfolgend berechnen wir Ihnen wie vorab besprochen:'],
 [33, 'Rechnung'],
 [34, '202210001'],
 [35, 'Kunde'],
 [36, '123456'],
 [37, 'Datum'],
 [38, '01.01.2023'],
 [39, 'USt-IDNr.'],
 [40, 'DE123456789'],
 [41, 'Steuer-Nr.'],
 [42, '122/5432/1234'],
 [43, 'Mobil'],
 [44, '01234/987654-2'],
 [45, 'Telefax'],
 [46, '01234/987654-1'],
 [47, 'Telefon'],
 [48, '01234/987654-0'],
 [49, 'Internet\nE-Mail'],
 [50, 'www.mustermann.de\nmax@muster.de'],
 [51, 'So erreichen Sie uns'],
 [52, 'Deutschland'],
 [53, 'Beispielstr. 27\n54657 Beispielstadt'],
 [54, 'Britta Beispiel'],
 [55, 'Frau'],
 [56, 'Max Mustermann - Musterstr. 1 - 41321 Musterstadt'],
 [57, 'Firmenlogo']]

In [264]:
bbox_coordinates

[{'bbox': [762, 3323, 585, 42]},
 {'bbox': [289, 3247, 310, 83]},
 {'bbox': [762, 3247, 291, 76]},
 {'bbox': [291, 2973, 329, 44]},
 {'bbox': [291, 2918, 454, 45]},
 {'bbox': [291, 2753, 1580, 108]},
 {'bbox': [288, 2642, 556, 53]},
 {'bbox': [1472, 2498, 363, 53]},
 {'bbox': [2013, 2498, 172, 50]},
 {'bbox': [1468, 2402, 718, 101]},
 {'bbox': [291, 2363, 1895, 20]},
 {'bbox': [289, 2283, 34, 44]},
 {'bbox': [468, 2283, 102, 44]},
 {'bbox': [701, 2283, 669, 53]},
 {'bbox': [1629, 2283, 25, 44]},
 {'bbox': [1842, 2283, 111, 50]},
 {'bbox': [2038, 2283, 147, 50]},
 {'bbox': [293, 2186, 25, 44]},
 {'bbox': [471, 2186, 99, 45]},
 {'bbox': [703, 2186, 650, 53]},
 {'bbox': [1505, 2186, 153, 50]},
 {'bbox': [1838, 2186, 115, 50]},
 {'bbox': [2035, 2186, 150, 50]},
 {'bbox': [291, 2090, 85, 44]},
 {'bbox': [466, 2090, 141, 44]},
 {'bbox': [691, 2090, 268, 53]},
 {'bbox': [1518, 2090, 141, 53]},
 {'bbox': [1724, 2090, 229, 52]},
 {'bbox': [2043, 2090, 141, 53]},
 {'bbox': [291, 2020, 1895, 20]}

In [265]:
def create_clusters(details_content , bbox_coordinates):
    for i in range(len(details_content)):
        bbox_x_coord = bbox_coordinates[details_content[i][0]]['bbox'][0]
        bbox_y_coord = bbox_coordinates[details_content[i][0]]['bbox'][1]
        details_content[i].append(bbox_x_coord)
        details_content[i].append(bbox_y_coord)
    
    details_content.sort(key = lambda x : x[-2])

    # combine all the details with same x coordinate
    clusters = []
    cluster = []
    cluster.append(details_content[0])
    prev_bbox_x_coord = details_content[0][-2]
    for i in range(1, len(details_content)):
        if abs(details_content[i][-2] - prev_bbox_x_coord)<50:
            cluster.append(details_content[i])
        else:
            cluster.sort(key = lambda x : x[-1])
            clusters.append(cluster)
            cluster = [details_content[i]]
        prev_bbox_x_coord = details_content[i][-2]
    
    cluster.sort(key = lambda x : x[-1])
    clusters.append(cluster)
    return clusters

x = details_content
clusters = create_clusters(x, bbox_coordinates)

In [266]:
clusters

[[[56, 'Max Mustermann - Musterstr. 1 - 41321 Musterstadt', 289, 458],
  [55, 'Frau', 291, 545],
  [54, 'Britta Beispiel', 291, 601],
  [53, 'Beispielstr. 27\n54657 Beispielstadt', 290, 656],
  [52, 'Deutschland', 291, 766],
  [32,
   'Sehr geehrte Frau Beispiel,\nnachfolgend berechnen wir Ihnen wie vorab besprochen:',
   290,
   1676],
  [31, 'Rechnung 202210001', 292, 1884],
  [30, 'Das Rechnungsdatum entspricht dem Leistungsdatum', 291, 1947],
  [29, '', 291, 2020]],
 [[51, 'So erreichen Sie uns', 1489, 863],
  [49, 'Internet\nE-Mail', 1491, 918],
  [47, 'Telefon', 1489, 1028],
  [45, 'Telefax', 1489, 1083],
  [43, 'Mobil', 1491, 1138],
  [41, 'Steuer-Nr.', 1490, 1248],
  [39, 'USt-IDNr.', 1491, 1303],
  [37, 'Datum', 1491, 1414],
  [35, 'Kunde', 1491, 1469],
  [33, 'Rechnung', 1491, 1524]],
 [[57, 'Firmenlogo', 1604, 74]],
 [[50, 'www.mustermann.de\nmax@muster.de', 1788, 918],
  [48, '01234/987654-0', 1790, 1028],
  [46, '01234/987654-1', 1790, 1083],
  [44, '01234/987654-2', 1790,

In [269]:
INVOICE_DETAILS = '''
You are given the data related to invoice... It could be related to seller, buyer, invoice number, date etc.

EXTRACTED DATA :
{extracted_data}

------------------------------------------------------
INSTRUCTIONS :
- Return a dictionary list of dictionaries 
- For each dictionary in the list, (keys and values) should be extracted from above extracted data.
- If extracted data has no information regarding key... Use logical assumptions to get the key for the data.
        # For example : "02.02.2022" can be assumed as date
        # Don't create keys for which value is not present in the extracted data.
        
- Return a dictionary with a key describing the list content in short and value as the list of dictionaries.
- Don't put the tag ---> ```python , while returning the answer.
ANSWER :
'''


def get_granular_invoice_details(clusters):
    li = []
    for cluster in clusters:
        cluster_text = ''
        for x in cluster:
            cluster_text+=f"- {x[1]}\n"
        prompt = PromptTemplate(INVOICE_DETAILS).format(extracted_data = cluster_text)
        response, _ = use_llm(prompt)
        # print(prompt , end = '\n\n\n\n')
        # print(response)
        # return li
        try : 
            response = ast.literal_eval(response)
            li.append(response)
        except:
            print(f'Response of llm : {response} for cluster : {cluster_text}' , '-'*100)
    return li

li = get_granular_invoice_details(clusters)
print(li)
    

Response of llm : {
 "Invoice_Data": [
  {
   "Contact_Details": "So erreichen Sie uns",
   "Contact_Mode": "Internet, E-Mail, Telefon, Telefax, Mobil",
   "Tax_Number": null,
   "VAT_Number": null,
   "Date": null,
   "Customer": null,
   "Invoice_Number": null
  }
 ]
} for cluster : - So erreichen Sie uns
- Internet
E-Mail
- Telefon
- Telefax
- Mobil
- Steuer-Nr.
- USt-IDNr.
- Datum
- Kunde
- Rechnung
 ----------------------------------------------------------------------------------------------------
[{'Invoice_Data': [{'Seller_Name': 'Max Mustermann', 'Seller_Address': 'Musterstr. 1 - 41321 Musterstadt', 'Buyer_Name': 'Britta Beispiel', 'Buyer_Address': 'Beispielstr. 27\n54657 Beispielstadt\nDeutschland', 'Invoice_Number': '202210001', 'Invoice_Date': None}]}, {'Invoice_Data': [{'Seller_Name': 'Firmenlogo'}]}, {'Invoice_Data': [{'Seller_Website': 'www.mustermann.de', 'Seller_Email': 'max@muster.de', 'Invoice_Number': '123456', 'Invoice_Date': '01.01.2023', 'Invoice_ID': '202210001'

In [268]:
with open(f'extracted_invoices/{file_name}.json', 'w') as json_file:
    dict_ = {
    'invoice_details' : li , 
    'product_details' : product_details
    }
    # print(dict_)
    json.dump(dict_, json_file, indent=4)

